# Create calculation schemes for VASP

Create defined VASP calculation schemes using the `CalculationSchemes` class in the `vasp.calculation_schemes` module. This class contains pre-defined schemes to perform different types of calculations. The starting point is a `Structure` object and `Incar` and `Kpoints` objects from `pymatgen`. These will define the basis of inputs for the scheme, where only the relavant INCAR tags and KPOINTS (where needed) will be changed according to the different scheme steps. Only the `Structure` argument is needed, if all other arguments are missing the objects will be constructed according to the `DefaultInputs` class.

### Define structure

We will retrieve the structure of Silicon from the Materials Project database using `pymatgen`.

In [2]:
from pynter.tools.materials_project import MPDatabase

structure = MPDatabase('mp-149').get_structure()
print(structure)

Full Formula (Si2)
Reduced Formula: Si
abc   :   3.839943   3.839943   3.839943
angles:  60.000000  60.000000  60.000000
Sites (2)
  #  SP        a      b      c
---  ----  -----  -----  -----
  0  Si    0.875  0.875  0.875
  1  Si    0.125  0.125  0.125


### Define set of inputs 

now we need to define a set of common inputs for the calculations. In this case we explicitely use the `DefaultInputs` class to show how it works, but if the default inputs are needed one can just omit the arguments when defining the `CalculationSchemes` class. 

In [6]:
from pynter.vasp.default_inputs import DefaultInputs
from pynter.vasp.calculation_schemes import CalculationSchemes

di = DefaultInputs(structure)
incar_settings = di.get_incar_default(xc='PBE')
kpoints = di.get_kpoints_default(kppa=1000)
# Get Pymatgen Potcar object. The PMG_VASP_PSP directory has to be defined in .pmgrc.yaml file as required by Pymatgen.
potcar = di.get_potcar(potcar_functional='PBE')

print(incar_settings)
print(kpoints)

{'IBRION': 2, 'NSW': 0, 'ISIF': 2, 'EDIFFG': -0.05, 'ISPIN': 1, 'LWAVE': '.TRUE.', 'LCHARG': '.TRUE.', 'LORBIT': 10, 'ENCUT': 500, 'EDIFF': 1e-06, 'ISMEAR': 0, 'SIGMA': 0.05, 'ALGO': 'Normal', 'AMIX': 0.2, 'LREAL': '.FALSE.', '#### Default PBE: system': 'Si', 'ISYM': 2}
pymatgen 4.7.6+ generated KPOINTS with grid density = 1000 / atom
0
Gamma
8 8 8



### Create CalculationSchemes object

Now we can create the `CalculationSchemes` object from the previously defined inputs. The argument `job_settings` is None by default, which means that the default settings for the job script are retrieved from the `config.yml` file when the input files are written.

In [8]:
cs = CalculationSchemes(structure,incar_settings,kpoints,potcar,job_settings=None,name='Si')

### Create Scheme object

Now we can use on of the methods of this class to select the desired scheme and create a `Scheme` object, which is constructed from a list of `Step` object, each made from a Pymatgen `VaspInput` object and a dictionary with job settings used by the `ScriptHandler` class in `slurm.job_script`. As an example we pick the `pbe_electronic_structure` scheme, which is made by a first electronic minimization step, followed by a DOS calculation with the number of kpoints tripled by default in every direction, and a non-sc BS calculation, with k-path build according to the defaults paths in Pymatgen.

In [10]:
scheme = cs.pbe_electronic_structure?

In [13]:
scheme = cs.pbe_electronic_structure();
scheme

Scheme object: 
[Step 1-PBE-SCF, Step 2-PBE-DOS, Step 3-PBE-BS]

In [19]:
# Check input params
for step in scheme.steps:
    print(step.name+':\n')
    print(step.incar)
    print(step.kpoints)
    print(step.job_settings)
    print('')

1-PBE-SCF:

#### Default PBE: system = Si
ALGO = Normal
AMIX = 0.2
EDIFF = 1e-06
EDIFFG = -0.05
ENCUT = 500
IBRION = 2
ISIF = 2
ISMEAR = 0
ISPIN = 1
ISYM = 2
LCHARG = .TRUE.
LORBIT = 10
LREAL = .FALSE.
LWAVE = .TRUE.
NSW = 0
SIGMA = 0.05

pymatgen 4.7.6+ generated KPOINTS with grid density = 1000 / atom
0
Gamma
8 8 8

{'name': 'Si_PBE-el-str_1'}

2-PBE-DOS:

#### Default PBE: system = Si
ALGO = Normal
AMIX = 0.2
EDIFF = 1e-06
EDIFFG = -0.05
ENCUT = 500
IBRION = 2
ICHARG = 1
ISIF = 2
ISMEAR = -5
ISPIN = 1
ISTART = 1
ISYM = 2
LCHARG = .TRUE.
LORBIT = 10
LREAL = .FALSE.
LWAVE = .TRUE.
NEDOS = 2000
NSW = 0
SIGMA = 0.05

Default gamma
0
Gamma
24 24 24

{'name': 'Si_PBE-el-str_2'}

3-PBE-BS:

#### Default PBE: system = Si
ALGO = Normal
AMIX = 0.2
EDIFF = 1e-06
EDIFFG = -0.05
ENCUT = 500
IBRION = 2
ICHARG = 11
ISIF = 2
ISMEAR = 0
ISPIN = 1
ISTART = 1
ISYM = 2
LCHARG = .TRUE.
LORBIT = 11
LREAL = .FALSE.
LWAVE = .TRUE.
NEDOS = 2000
NSW = 0
SIGMA = 0.05

Line_mode KPOINTS file
10
Line_mode
Recip

### Write Scheme to files

Now we can write all the created input parameters to files to directly perform the calculations. Alternatively they can be used to create `VaspJob` objects with the `create_job` method in `data.datasets.Dataset` to run the calculations directly from the jupyter notebook.

In [18]:
scheme.write_scheme(path='./Si-BS',make_dir_if_not_present=True)